In [2]:
import copy
import json
from pathlib import Path
import requests
import numpy as np
import pandas as pd
from gspread_pandas import Spread, Client, conf
from trueskill import Rating, rate

In [3]:
spread = Spread('1tkX0iQF1tkrBSNVaEd4nAlisFt3AlwWGG0VrG8_OMPw', config=conf.get_config(conf_dir=Path.cwd()))

In [49]:
players_df = spread.sheet_to_df(index=0, sheet='players')
games_df = spread.sheet_to_df(index=0, sheet='game_ids')
player_participants_df = spread.sheet_to_df(index=0, sheet='game_participant_ids').astype({"participant_id":int})
champions_df = spread.sheet_to_df(index=0, sheet='champions').astype({"champion_id":int})

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": "RGAPI-665dec09-0847-4ca8-8fb2-eac58184f622"
}

player_stats = list()
inhouse_counter = 0
for g in games_df['league_game_id'].unique():
    r = requests.get(f'https://na1.api.riotgames.com/lol/match/v4/matches/{g}', headers=headers)
    for p in r.json()['participants']:
        try:
            cs_diff_per_min_deltas = p['timeline']['csDiffPerMinDeltas']
        except:
            cs_diff_per_min_deltas = np.nan
            
        try:
            xp_diff_per_min_deltas = p['timeline']['xpDiffPerMinDeltas']
        except:
            xp_diff_per_min_deltas = np.nan
            
        try:
            creeps_per_min_deltas = p['timeline']['creepsPerMinDeltas']
        except:
            creeps_per_min_deltas = np.nan
        player_stats.append([g, 
                             p['participantId'], 
                             inhouse_counter,
                             p['teamId'], 
                             p['championId'],
                             r.json()['gameDuration'],
                             p['stats']['win'], 
                             p['stats']['kills'],
                             p['stats']['deaths'],
                             p['stats']['assists'],
                             p['stats']['totalDamageDealtToChampions'],
                             p['stats']['totalDamageTaken'],
                             p['stats']['totalMinionsKilled'],
                             p['stats']['neutralMinionsKilled'],
                             p['timeline']['role'],
                             p['timeline']['lane'],
                             cs_diff_per_min_deltas,
                             xp_diff_per_min_deltas,  
                             creeps_per_min_deltas
                            ])
    inhouse_counter += 1

player_stats_df = pd.DataFrame(player_stats, 
                               columns=['game_id', 
                                        'participant_id', 
                                        'inhouse_id',
                                        'team_id', 
                                        'champion_id',
                                        'game_duration',
                                        'win',
                                        'kills',
                                        'deaths',
                                        'assists',
                                        'total_damage_dealt_to_champs',
                                        'total_damage_taken',
                                        'minions_killed',
                                        'creeps_killed',
                                        'role',
                                        'lane',
                                        'cs_diff_per_min_delta',
                                        'xp_diff_per_min_delta',
                                        'creeps_per_min'
                                       ])

In [50]:
deltas = ['cs_diff_per_min_delta', 'xp_diff_per_min_delta', 'creeps_per_min']
for c in deltas:
    player_stats_df = player_stats_df.merge(
        player_stats_df[c]
        .apply(pd.Series)
        .rename(columns={"0-10":f'{c}_0-10', "10-20" : f'{c}_10-20', "20-30":f'{c}_20-30', "30-end":f'{c}_30-end'}), 
        left_index=True, 
        right_index=True
    )
    

In [51]:
role_dict = {
    "BOTTOM/DUO_CARRY":"BOT", 
    "JUNGLE/NONE":"JUNGLE", 
    "TOP/SOLO":"TOP", 
    "MIDDLE/SOLO":"MIDDLE", 
    "TOP/DUO":"TOP", 
    "MIDDLE/DUO":"MIDDLE",     
    "BOTTOM/DUO_SUPPORT":"SUPPORT"
}

In [52]:
player_stats_df = (
    player_stats_df
    .merge(player_participants_df)
    .merge(players_df)
    # This breaks on new champions, I don't know if there's a great fix
    .merge(champions_df)
    .assign(
        combined_role = lambda d: (d['lane'] + '/' + d['role']),
        win = lambda d: d['win'].astype(int),
        role = lambda d: d['combined_role'].map(role_dict)
           )
    .drop(columns=['participant_id', 'cs_diff_per_min_delta', 'xp_diff_per_min_delta', 'creeps_per_min', 'lane'])
)

In [54]:
post_game_ratings = list()
player_ratings = {}
counter = 0

players = player_stats_df['player_id'].unique()
for player in players:
    player_ratings[player] = Rating()

for match in sorted(player_stats_df['inhouse_id'].unique()):
    match_df = player_stats_df[player_stats_df['inhouse_id']==match]
    # this feels really dumb but for each match, I need to know who was on what team and to maintain order
    ratings_1 = list()
    ratings_2 = list()
    team_1 = list()
    team_2 = list()
    for player in match_df['player_id']:
        if match_df[match_df['player_id'] == player]['win'].values[0] == 1:
            ratings_1.append(player_ratings[player])
            team_1.append(player)
        else:
            ratings_2.append(player_ratings[player])
            team_2.append(player)
    try:
        new_r1, new_r2 = rate([ratings_1, ratings_2], ranks=[0, 1])
    except:
        # At some point you will incorrectly enter in the players, this will help you troubleshoot
        print(match, team1, team2)

    for i in range(0,5):
        player_ratings[team_1[i]] = new_r1[i]
        player_ratings[team_2[i]] = new_r2[i]
        
    for player in players:
        post_game_ratings.append([counter, player, player_ratings[player].sigma, player_ratings[player].mu])
    counter += 1        


current_elos_df = pd.DataFrame.from_dict(player_ratings, orient='index', columns=['mu', 'sigma']).reset_index().rename(columns={"index":"player_id"})

per_game_elo_df = pd.DataFrame(post_game_ratings, columns=['game_id', 'player_id', 'sigma', 'mu'])

output_df = (
    player_stats_df
    .groupby(['player_id'])
    .agg({"game_id":"nunique", "win":"sum"})
    .reset_index()
    .merge(current_elos_df)
    .merge(players_df)
    .rename(columns={"game_id":"games_played", "win":"games_won", "mu":"elo", "sigma":"elo_variance"})
    .loc[:,['name', 'ign', 'elo', 'elo_variance','games_played', 'games_won']]
    .sort_values('elo', ascending=False)
)


spread.df_to_sheet(df=output_df, sheet='stats')

spread.df_to_sheet(df=per_game_elo_df, sheet='elo_over_time')

spread.df_to_sheet(df=player_stats_df, sheet='player_stats')

0
['6', '4', '0', '12', '9'] ['11', '10', '1', '2', '3']
1
['10', '1', '4', '5', '9'] ['11', '3', '2', '0', '6']
2
['1', '2', '7', '9', '5'] ['11', '10', '12', '13', '6']
3
['13', '1', '2', '6', '7'] ['11', '5', '12', '4', '9']
4
['11', '5', '4', '6', '13'] ['3', '0', '2', '7', '9']
5
['5', '4', '13', '0', '7'] ['11', '3', '2', '6', '9']
6
['10', '3', '8', '6', '13'] ['11', '12', '4', '5', '7']
7
['11', '12', '6', '0', '7'] ['10', '2', '5', '4', '13']
8
['10', '6', '4', '7', '0'] ['11', '3', '13', '12', '5']
9
['2', '13', '4', '6', '5'] ['11', '10', '0', '3', '12']
10
['11', '4', '0', '13', '5'] ['10', '2', '3', '7', '8']
11
['3', '1', '4', '13', '6'] ['12', '11', '2', '7', '0']
12
['4', '6', '8', '12', '5'] ['11', '1', '2', '3', '0']
13
['0', '12', '13', '9', '5'] ['11', '1', '2', '3', '4']
14
['11', '1', '12', '6', '13'] ['5', '3', '4', '9', '7']
15
['5', '0', '2', '13', '7'] ['11', '12', '1', '6', '4']
16
['12', '6', '1', '4', '9'] ['11', '2', '0', '7', '5']
17
['6', '12', '0', '9',

In [55]:
# This is all network graph stuff
team_wins = (
    player_stats_df[['game_id', 'team_id', 'win', 'ign']]
    .merge(
        player_stats_df[['game_id', 'team_id', 'ign']],
        left_on=['game_id', 'team_id'],
        right_on=['game_id', 'team_id']
    )
    .astype({"win":int})
    .groupby(['ign_x', 'ign_y'])
    .agg({"game_id":"nunique", "win":"sum"})
    .reset_index()
    .rename(columns={"game_id":"teammates_games_played", "win":"teammates_games_won"})
)


h2h_wins = (
    player_stats_df[['game_id', 'team_id', 'win', 'ign']]
    .merge(
        player_stats_df[['game_id', 'team_id', 'ign']],
        left_on=['game_id'],
        right_on=['game_id']
    )
    .loc[lambda d: d['team_id_x'] != d['team_id_y'], :]
    .astype({"win":int})
    .groupby(['ign_x', 'ign_y'])
    .agg({"game_id":"nunique", "win":"sum"})
    .reset_index()
    .rename(columns={"game_id":"h2h_games_played", "win":"h2h_games_won"})
)

network_graph_df = (
    team_wins
    .merge(h2h_wins, how='outer')
    .assign(direction = lambda d: d['ign_x'] + '->' + d['ign_y'])
    .rename(columns={"ign_x":"player_1", "ign_y":"player_2"})
    .groupby(['player_1', 'player_2', 'direction'])
    .agg({"teammates_games_played":"max", "teammates_games_won":"max", "h2h_games_played":"max", "h2h_games_won":"max"})
    .reset_index()
    .assign(X = lambda d: np.random.randint(-300, 300, d.shape[0]),
            Y = lambda d: np.random.randint(-300, 300, d.shape[0])
           )
)

dummy_df = (
    network_graph_df
    .assign(X = 0,
           Y = 0)
)

network_graph_df = network_graph_df.append(dummy_df)

spread.df_to_sheet(df=network_graph_df, sheet='network_graph')